In [ ]:
## vscode 

# python: 인터프리터 선택 > conda env 동일하게 선택

In [ ]:
# !pip install torch torchvision

In [ ]:
# imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

# from torch.utils.tensorboard import SummaryWriter

import torchvision
from torchvision import transforms, datasets

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

print("Using PyTorch version:", torch.__version__,' Device:', device)

In [ ]:
# transforms
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

# datasets
train_set = datasets.FashionMNIST('./data',
    download=True,
    train=True,
    transform=transform)
test_set = datasets.FashionMNIST('./data',
    download=True,
    train=False,
    transform=transform)

# dataloaders
BATCH_SIZE = 32

train_loader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
# classes
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
        'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)
        
    def forward(self,x):
        x = x.view(-1, 28*28)
        x = self.fc1(x)
        x = F.sigmoid(x)
        x = self.fc2(x)
        x = F.sigmoid(x)
        x = self.fc3(x)
        x = F.log_softmax(x, dim=1)
        return x

In [ ]:
# Model
model = Net().to(device)

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Loss function
criterion = nn.CrossEntropyLoss()

# scheduler
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

print(model)

In [ ]:
# writer = SummaryWriter('runs/fashion_mnist_experiment')

In [ ]:
# def visualize_samples_tensorboard(loader, classes, samples_per_class=5):
#     class_dict = {i: [] for i in range(len(classes))}
    
#     for img, label in loader:
#         for i in range(len(img)):
#             if len(class_dict[label[i].item()]) < samples_per_class:
#                 class_dict[label[i].item()].append(img[i])
#         if all(len(v) == samples_per_class for v in class_dict.values()):
#             break
    
#     for label, images in class_dict.items():
#         grid = torchvision.utils.make_grid(images, nrow=samples_per_class, normalize=True)
#         writer.add_image(f'{classes[label]} samples', grid, 0)

# visualize_samples_tensorboard(train_loader, classes)

In [ ]:
# 훈련 함수
def train(model, train_loader, optimizer, scheduler, criterion, epoch, device, writer=None):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        # parameter initialization
        optimizer.zero_grad()
        
        # forward pass
        output = model(data)
        
        # calculate loss
        loss = criterion(output, target)
        
        # backward pass
        loss.backward()
        
        # optimize
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_accuracy = 100. * correct / total

    print(f'Train Epoch: {epoch} Loss: {epoch_loss:.4f} Accuracy: {epoch_accuracy:.2f}%')
    
    scheduler.step()
    # writer.add_scalar('Loss/train', epoch_loss, epoch)
    # writer.add_scalar('Accuracy/train', epoch_accuracy, epoch)

# 테스트 함수
def test(model, test_loader, criterion, epoch, device, writer=None):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            test_loss += loss.item()
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / total

    print(f'Test Epoch: {epoch} Loss: {test_loss:.4f} Accuracy: {test_accuracy:.2f}%')
    # writer.add_scalar('Loss/test', test_loss, epoch)
    # writer.add_scalar('Accuracy/test', test_accuracy, epoch)

# 훈련 및 테스트 실행
epochs = 50

for epoch in range(1, epochs + 1):
    train(model, train_loader, optimizer, scheduler, criterion, epoch, device)
    test(model, test_loader, criterion, epoch, device)

In [ ]:
# writer.close()